In [81]:
from datetime import date
from datetime import timedelta
import calendar
import schedule
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import dash
import dash_bootstrap_components as dbc
import dash_html_components as html
import dash_core_components as dcc
import plotly.express as px
from dash.dependencies import Input, Output
import dash_table as dt
import  psycopg2
from psycopg2 import Error
from sqlalchemy import text
from datetime import date
import schedule

query2 = open('Scripts/query2.sql')
stock = open('Scripts/user_stock.sql')
sup = open('Scripts/Supervisor.sql')
vente = open('Scripts/ventes.sql')
ticket = open('Scripts/ticket.sql')
backlog = open('Scripts/backlog.sql')


sql_text1 = query2.read()
sql_text2 = stock.read()
sql_text3 = vente.read()
sql_text4 = sup.read()
sql_text5 = ticket.read()
sql_text6 = backlog.read()

# Connection to oolusolar database
try:
    connection = psycopg2.connect(user = 'chartio_read_only_user',
                                  password = '2ZVF01USUWTKV3K9JJFY',
                                  host = 'oolu-main-postgresql.cfa4plgxjs0u.eu-central-1.rds.amazonaws.com',
                                  port ='5432',
                                  database = 'oolusolar_analytics')
    cursor = connection.cursor()
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print('You are Successfully connected to - ',record, '\n')
except (Exception, Error) as error:
    print(" Connection failed, try again", error)
    cursor.close()

df_ag_sup  = pd.read_sql_query(sql_text1 , connection)
df_stock  = pd.read_sql_query(sql_text2 , connection)
df_vente  = pd.read_sql_query(sql_text3 , connection)
df_sup  = pd.read_sql_query(sql_text4 , connection)
df_backlog  = pd.read_sql_query(sql_text6 , connection)
df_ticket  = pd.read_sql_query(sql_text5 , connection)

cursor.close()
connection.close()


table_header_style = {
    "backgroundColor": "rgb(2,21,70)",
    "color": "white",
    "textAlign": "center",
    'fontSize': 20
}

You are Successfully connected to -  ('PostgreSQL 9.5.24 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 



In [82]:
df_ag_sup.head()

,agent_id,date_debut,username,agent,agent_role,role_id,zone_id,zone,id_tenant,superviseur,sup_username,sup_id
0,3671,2020-12-03,saintlouispoly02,MAMADOU DIAGNE,Agent Polivalent,5,32,SAINT LOUIS,1,KHASSIM DIOP,khassim.diop@oolusolar.com,60
1,2617,2019-10-21,kedougoupoly09,Mamadou Kaba Diaby,Agent Polivalent,5,35,KEDOUGOU,1,DEMBA GUEYE,demba.gueye,2440
2,2929,2020-01-14,guindo,Seydou GUINDO,Agent Commercial,14,16,Zone houndé,4,Emmanuel KABRE,emmanuel.kabre,153
3,2618,2019-10-21,kedougoupoly10,Boye Sara Diallo,Agent Polivalent,5,35,KEDOUGOU,1,DEMBA GUEYE,demba.gueye,2440
4,1303,2018-11-27,agentcom82,Mbaye Dia,Agent Commercial,14,30,LOUGA,1,THIERNO FAYE,thierno.faye@oolusolar.com,1561


In [83]:
df_stock.head()

,stock_name,stock_type,agent_id,product_type,category,stock
0,Abba Goita - abba.goita,personal,190,Fan component,None,1
1,Abba Goita - abba.goita,personal,190,SKH Controller,pico,3
2,Abba Goita - abba.goita,personal,190,WowSolar Controller,pico,11
3,ABBEY BLESSED - EKPARAKWA01,personal,3141,Burn stove component,accessories,2
4,ABBEY BLESSED - EKPARAKWA01,personal,3141,Fan component,None,4


In [84]:
df_ag_sup.head()

,agent_id,date_debut,username,agent,agent_role,role_id,zone_id,zone,id_tenant,superviseur,sup_username,sup_id
0,3671,2020-12-03,saintlouispoly02,MAMADOU DIAGNE,Agent Polivalent,5,32,SAINT LOUIS,1,KHASSIM DIOP,khassim.diop@oolusolar.com,60
1,2617,2019-10-21,kedougoupoly09,Mamadou Kaba Diaby,Agent Polivalent,5,35,KEDOUGOU,1,DEMBA GUEYE,demba.gueye,2440
2,2929,2020-01-14,guindo,Seydou GUINDO,Agent Commercial,14,16,Zone houndé,4,Emmanuel KABRE,emmanuel.kabre,153
3,2618,2019-10-21,kedougoupoly10,Boye Sara Diallo,Agent Polivalent,5,35,KEDOUGOU,1,DEMBA GUEYE,demba.gueye,2440
4,1303,2018-11-27,agentcom82,Mbaye Dia,Agent Commercial,14,30,LOUGA,1,THIERNO FAYE,thierno.faye@oolusolar.com,1561


In [85]:
df_ag_sup.dtypes

agent_id         int64
date_debut      object
username        object
agent           object
agent_role      object
role_id          int64
zone_id          int64
zone            object
id_tenant        int64
superviseur     object
sup_username    object
sup_id           int64
dtype: object

In [86]:
df_vente.head()

,created_at,annee,mois,jour,agent_id,zone_id,agent,id_tenant,group_prix,deposit,ventes
0,2019-07-24,2019,7,24,1480,21.0,Germain Fayama,4,WowSolarTV19 - Cash - BF,220000,1
1,2018-04-01,2018,4,1,291,9.0,Bagadian Keita,2,PAYG 9000/4500,9000,1
2,2020-01-09,2020,1,9,360,9.0,Abdoulaye Keita,2,WowSolarTV22 - Mensuel - ML,55500,1
3,2019-10-15,2019,10,15,1271,26.0,ALMAMY SAMORY SAGNA,1,WowSolar60_Mensuelle 8500/4250,8500,1
4,2019-01-21,2019,1,21,240,38.0,MAMADOU DIALLO agentcom122,1,Vente Annuelle (41500 x 2),41500,1


In [88]:
df_backlog.head()

,agent_id,agent,type_ticket,backlog
0,2862,KADIALY MORO DANGNOKHO,Reposséder,2
1,2041,Abdoulaye Bassolé,Ticket technique,1
2,501,dorcas adeoti,Reposséder,12
3,215,PAPE IBRAHIMA FAYE,Ticket technique,69
4,1498,Arsene koulade,Ticket technique,5


In [11]:
df_vente['created_at']= pd.to_datetime(df_vente['created_at'])
df_ag_sup['date_debut']= pd.to_datetime(df_ag_sup['date_debut'])
df_backlog['closed_date']= pd.to_datetime(df_backlog['closed_date'])
df_solved['solved_at']= pd.to_datetime(df_solved['solved_at'])

print(df_vente['created_at'].dtype)

datetime64[ns]


In [12]:
# df_vente['day_of_week'] = df_vente['created_at'].apply(lambda x: x.weekday()+1)
# df_vente.head()

In [13]:
df_ag_stock = pd.merge(df_ag_sup,df_stock, how= 'left', on= 'agent_id')
print(df_ag_stock.shape)
df_ag_stock.head()

(1382, 17)


,agent_id,date_debut,username,agent,agent_role,role_id,zone_id,zone,id_tenant,superviseur,sup_username,sup_id,stock_name,stock_type,product_type,category,stock
0,3671,2020-12-03,saintlouispoly02,MAMADOU DIAGNE,Agent Polivalent,5,32,SAINT LOUIS,1,KHASSIM DIOP,khassim.diop@oolusolar.com,60,NaN,NaN,NaN,NaN,NaN
1,2617,2019-10-21,kedougoupoly09,Mamadou Kaba Diaby,Agent Polivalent,5,35,KEDOUGOU,1,DEMBA GUEYE,demba.gueye,2440,NaN,NaN,NaN,NaN,NaN
2,2929,2020-01-14,guindo,Seydou GUINDO,Agent Commercial,14,16,Zone houndé,4,Emmanuel KABRE,emmanuel.kabre,153,NaN,NaN,NaN,NaN,NaN
3,2618,2019-10-21,kedougoupoly10,Boye Sara Diallo,Agent Polivalent,5,35,KEDOUGOU,1,DEMBA GUEYE,demba.gueye,2440,NaN,NaN,NaN,NaN,NaN
4,1303,2018-11-27,agentcom82,Mbaye Dia,Agent Commercial,14,30,LOUGA,1,THIERNO FAYE,thierno.faye@oolusolar.com,1561,NaN,NaN,NaN,NaN,NaN


In [14]:
agent_feat = ['agent','date_debut','agent_role','stock_type','category','superviseur','zone','stock']
df_ag_stock = df_ag_stock[agent_feat]
stock_ag = df_ag_stock.groupby(['agent','date_debut','agent_role','superviseur','zone']).sum().reset_index()
stock_ag.head()

,agent,date_debut,agent_role,superviseur,zone,stock
0,Bernard DRABO,2019-09-12,Agent Commercial,Sahiba DIAO,Zone Nayala,0.0
1,Jathet Mounkoro,2020-09-09,Agent Commercial,Koita Daouda,SAN,0.0
2,ABDOU MBOW,2019-07-30,Agent Polivalent,AMADOU ABOU SALL,MATAM,0.0
3,ABDOU RAZACK BALDE,2018-08-13,Agent Polivalent,IBRAHIMA THIONGANE,KOLDA,2.0
4,ABDOUL AZIZ DRABO,2020-05-29,Agent Polivalent,Arouna BADO,Zone Cascades,0.0


In [15]:
df_vente_agent = pd.merge(df_vente,df_ag_sup[['agent_id','superviseur','zone']], how= 'inner', on= 'agent_id')

In [16]:
df_vente_agent.head()

,created_at,annee,mois,jour,agent_id,zone_id,agent,id_tenant,group_prix,deposit,ventes,superviseur,zone
0,2019-07-24,2019,7,24,1480,21.0,Germain Fayama,4,WowSolarTV19 - Cash - BF,220000,1,Arouna BADO,Zone Cascades
1,2019-05-02,2019,5,2,1480,21.0,Germain Fayama,4,WowSolarTV19 - Annuel - BF,130000,1,Arouna BADO,Zone Cascades
2,2019-06-22,2019,6,22,1480,21.0,Germain Fayama,4,WowSolarTV19 - Mensuel - BF,55500,1,Arouna BADO,Zone Cascades
3,2019-01-19,2019,1,19,1480,21.0,Germain Fayama,4,WowSolar60 - Annuel - BF,35000,1,Arouna BADO,Zone Cascades
4,2020-02-01,2020,2,1,1480,21.0,Germain Fayama,4,WowSolarTV32 - Annuel - BF,150000,1,Arouna BADO,Zone Cascades


In [17]:
def week_int2(x):
    y =x.year
    m = x.month
    d = x.day
    list_week = calendar.monthcalendar(y,m)
    for i in range(len(list_week)):
        if d in list_week[i]:
            return 'week_'+str(i+1)

In [18]:
df_vente_agent['Weeks'] = df_vente_agent['created_at'].apply(week_int2)
df_vente_agent['Month'] = df_vente_agent['created_at'].apply(lambda x: x.strftime('%b'))
df_vente_agent['Day'] = df_vente_agent['created_at'].apply(lambda x: x.strftime('%a'))
df_vente_agent['CA'] = df_vente_agent['deposit']*df_vente_agent['ventes']

In [19]:
df_vente_agent.head()

,created_at,annee,mois,jour,agent_id,zone_id,agent,id_tenant,group_prix,deposit,ventes,superviseur,zone,Weeks,Month,Day,CA
0,2019-07-24,2019,7,24,1480,21.0,Germain Fayama,4,WowSolarTV19 - Cash - BF,220000,1,Arouna BADO,Zone Cascades,week_4,Jul,Wed,220000
1,2019-05-02,2019,5,2,1480,21.0,Germain Fayama,4,WowSolarTV19 - Annuel - BF,130000,1,Arouna BADO,Zone Cascades,week_1,May,Thu,130000
2,2019-06-22,2019,6,22,1480,21.0,Germain Fayama,4,WowSolarTV19 - Mensuel - BF,55500,1,Arouna BADO,Zone Cascades,week_4,Jun,Sat,55500
3,2019-01-19,2019,1,19,1480,21.0,Germain Fayama,4,WowSolar60 - Annuel - BF,35000,1,Arouna BADO,Zone Cascades,week_3,Jan,Sat,35000
4,2020-02-01,2020,2,1,1480,21.0,Germain Fayama,4,WowSolarTV32 - Annuel - BF,150000,1,Arouna BADO,Zone Cascades,week_1,Feb,Sat,150000


In [20]:
feature = ['created_at','annee','mois','agent','agent_id','group_prix','superviseur','zone','Weeks','Month','Day','CA','ventes']
df_vente_mois = df_vente_agent[feature][(df_vente_agent['annee']==date.today().year)&(df_vente_agent['mois']==date.today().month)]
df_vente_mois.head()

,created_at,annee,mois,agent,agent_id,group_prix,superviseur,zone,Weeks,Month,Day,CA,ventes
71,2021-02-12,2021,2,Bagadian Keita,291,WowSolar60 - Mensuel - ML,David Timbine,KAYES,week_2,Feb,Fri,18000,2
106,2021-02-14,2021,2,Bagadian Keita,291,WowSolar60 - Cash - ML,David Timbine,KAYES,week_2,Feb,Sun,60000,1
142,2021-02-12,2021,2,Bagadian Keita,291,WowSolar60 - Cash - ML,David Timbine,KAYES,week_2,Feb,Fri,60000,1
143,2021-02-07,2021,2,Bagadian Keita,291,WowSolar60 - Cash - ML,David Timbine,KAYES,week_1,Feb,Sun,60000,1
164,2021-02-22,2021,2,Bagadian Keita,291,WowSolar60 - Annuel - ML,David Timbine,KAYES,week_4,Feb,Mon,35000,1


In [21]:
week_ca = pd.pivot_table(df_vente_mois,values='CA',index=['agent_id','agent','superviseur'],columns='Weeks', aggfunc=np.sum).reset_index()
week_ca.fillna(0,inplace=True)
day_ca = pd.pivot_table(df_vente_mois,values='CA',index=['agent_id','agent','superviseur','Weeks'],columns='Day', aggfunc=np.sum).reset_index()
day_ca.fillna(0,inplace=True)
day_ca.head()

Day,agent_id,agent,superviseur,Weeks,Fri,Mon,Sat,Sun,Thu,Tue,Wed
0,45,ALPHA BABOU,Ndeye Saly Faye,week_1,52000.0,8500.0,0.0,0.0,140000.0,0.0,0.0
1,45,ALPHA BABOU,Ndeye Saly Faye,week_2,300000.0,0.0,0.0,0.0,0.0,0.0,0.0
2,53,AMADOU BOCAR SAGNA,SADIA TAMBA,week_1,0.0,0.0,0.0,0.0,0.0,140000.0,0.0
3,53,AMADOU BOCAR SAGNA,SADIA TAMBA,week_2,0.0,0.0,0.0,0.0,175000.0,0.0,41500.0
4,82,Abdoulaye Bamako Coulibaly,Samba Coulibaly,week_1,0.0,35000.0,0.0,0.0,0.0,9000.0,0.0


In [22]:
week_ca.head()

Weeks,agent_id,agent,superviseur,week_1,week_2,week_3,week_4
0,45,ALPHA BABOU,Ndeye Saly Faye,200500.0,300000.0,0.0,0.0
1,53,AMADOU BOCAR SAGNA,SADIA TAMBA,140000.0,216500.0,0.0,0.0
2,82,Abdoulaye Bamako Coulibaly,Samba Coulibaly,44000.0,0.0,0.0,0.0
3,114,NIOKHOR TINE,KHADIM SECK,286500.0,166500.0,265000.0,55000.0
4,190,Abba Goita,Karimou Traoré,0.0,18000.0,129000.0,0.0


In [23]:
yesterday = date.today()-timedelta(days=1)
print(week_int2(yesterday))
date.today().weekday()

week_4


3

In [24]:
feat_sales = ['agent_id','Mon','Tue', 'Wed','Thu', 'Sat', 'Sun']
df_day_ca = day_ca[feat_sales][day_ca['Weeks']==week_int2(date.today())]
df_day_ca.head()

Day,agent_id,Mon,Tue,Wed,Thu,Sat,Sun
8,114,55000.0,0.0,0.0,0.0,0.0,0.0
12,198,9000.0,55500.0,0.0,0.0,0.0,0.0
15,199,35000.0,0.0,0.0,0.0,0.0,0.0
17,200,9000.0,9000.0,0.0,0.0,0.0,0.0
35,230,8500.0,0.0,0.0,0.0,0.0,0.0


In [25]:
df_ca = pd.merge(week_ca,df_day_ca, how= 'left', on= 'agent_id')
df_ca.fillna(0,inplace=True)
df_ca = df_ca.drop([week_int2(date.today()),'agent_id'],axis=1)
df_ca.head()

,agent,superviseur,week_1,week_2,week_3,Mon,Tue,Wed,Thu,Sat,Sun
0,ALPHA BABOU,Ndeye Saly Faye,200500.0,300000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AMADOU BOCAR SAGNA,SADIA TAMBA,140000.0,216500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Abdoulaye Bamako Coulibaly,Samba Coulibaly,44000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NIOKHOR TINE,KHADIM SECK,286500.0,166500.0,265000.0,55000.0,0.0,0.0,0.0,0.0,0.0
4,Abba Goita,Karimou Traoré,0.0,18000.0,129000.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
df_ca.shape

(152, 11)

In [27]:
week_sales = pd.pivot_table(df_vente_mois,values='ventes',index=['agent_id','agent','superviseur','group_prix','zone'],columns='Weeks', aggfunc=np.sum).reset_index()
week_sales.fillna(0,inplace=True)

In [28]:
week_sales.head()

Weeks,agent_id,agent,superviseur,group_prix,zone,week_1,week_2,week_3,week_4
0,45,ALPHA BABOU,Ndeye Saly Faye,Ventilateur - Cash - SN,FATICK,1.0,0.0,0.0,0.0
1,45,ALPHA BABOU,Ndeye Saly Faye,WowSolar60_Mensuelle 8500/4250,FATICK,1.0,0.0,0.0,0.0
2,45,ALPHA BABOU,Ndeye Saly Faye,WowSolarTV22 - Annuel - SN,FATICK,1.0,0.0,0.0,0.0
3,45,ALPHA BABOU,Ndeye Saly Faye,WowSolarTV32 - Cash - SN,FATICK,0.0,1.0,0.0,0.0
4,53,AMADOU BOCAR SAGNA,SADIA TAMBA,WowSolar60_Annuelle 41500x2,SEDHIOU,0.0,1.0,0.0,0.0


In [29]:
list(week_sales.columns)

['agent_id',
 'agent',
 'superviseur',
 'group_prix',
 'zone',
 'week_1',
 'week_2',
 'week_3',
 'week_4']

In [30]:
days_sales = pd.pivot_table(df_vente_mois,values='ventes',index=['agent_id','agent','superviseur','group_prix','zone','Weeks'],columns='Day', aggfunc=np.sum).reset_index()
days_sales.fillna(0,inplace=True)
days_sales.head()

Day,agent_id,agent,superviseur,group_prix,zone,Weeks,Fri,Mon,Sat,Sun,Thu,Tue,Wed
0,45,ALPHA BABOU,Ndeye Saly Faye,Ventilateur - Cash - SN,FATICK,week_1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,45,ALPHA BABOU,Ndeye Saly Faye,WowSolar60_Mensuelle 8500/4250,FATICK,week_1,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,45,ALPHA BABOU,Ndeye Saly Faye,WowSolarTV22 - Annuel - SN,FATICK,week_1,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,45,ALPHA BABOU,Ndeye Saly Faye,WowSolarTV32 - Cash - SN,FATICK,week_2,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,53,AMADOU BOCAR SAGNA,SADIA TAMBA,WowSolar60_Annuelle 41500x2,SEDHIOU,week_2,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [31]:
days_sales.columns

Index(['agent_id', 'agent', 'superviseur', 'group_prix', 'zone', 'Weeks',
       'Fri', 'Mon', 'Sat', 'Sun', 'Thu', 'Tue', 'Wed'],
      dtype='object', name='Day')

In [32]:
feat_sales = ['agent_id','group_prix','Mon','Tue', 'Wed','Thu', 'Sat', 'Sun']
df_daysales = days_sales[feat_sales][days_sales['Weeks']==week_int2(date.today())]
df_daysales.head()

Day,agent_id,group_prix,Mon,Tue,Wed,Thu,Sat,Sun
15,114,WowSolarTV22 - Mensuel - SN,1.0,0.0,0.0,0.0,0.0,0.0
23,198,WowSolar60 - Mensuel - ML,1.0,0.0,0.0,0.0,0.0,0.0
24,198,WowSolarTV22 - Mensuel - ML,0.0,1.0,0.0,0.0,0.0,0.0
25,199,WowSolar60 - Annuel - ML,1.0,0.0,0.0,0.0,0.0,0.0
29,200,WowSolar60 - Mensuel - ML,1.0,1.0,0.0,0.0,0.0,0.0


In [33]:
df_vente = pd.merge(week_sales,df_daysales, how= 'left', on= ['agent_id','group_prix'])
df_vente.fillna(0,inplace=True)
df_vente = df_vente.drop([week_int2(date.today()),'agent_id'],axis=1)
df_vente.head()

,agent,superviseur,group_prix,zone,week_1,week_2,week_3,Mon,Tue,Wed,Thu,Sat,Sun
0,ALPHA BABOU,Ndeye Saly Faye,Ventilateur - Cash - SN,FATICK,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ALPHA BABOU,Ndeye Saly Faye,WowSolar60_Mensuelle 8500/4250,FATICK,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ALPHA BABOU,Ndeye Saly Faye,WowSolarTV22 - Annuel - SN,FATICK,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ALPHA BABOU,Ndeye Saly Faye,WowSolarTV32 - Cash - SN,FATICK,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AMADOU BOCAR SAGNA,SADIA TAMBA,WowSolar60_Annuelle 41500x2,SEDHIOU,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
df_vente.columns

Index(['agent', 'superviseur', 'group_prix', 'zone', 'week_1', 'week_2',
       'week_3', 'Mon', 'Tue', 'Wed', 'Thu', 'Sat', 'Sun'],
      dtype='object')

In [35]:
total = df_vente.groupby(['agent', 'superviseur']).sum().reset_index()
total.head()

,agent,superviseur,week_1,week_2,week_3,Mon,Tue,Wed,Thu,Sat,Sun
0,ABDOU RAZACK BALDE,IBRAHIMA THIONGANE,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,ABDOURAHMANI ZIO,Y. Clément YONLI,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,ABOU SARR,AMADOU ABOU SALL,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ADAMA DIALLO,SADIO CISSOKHO,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ALIOU CISSE,SADIA TAMBA,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
total['group_prix'] = ['Total' for i in range(len(total))]
# feat_total = ['agent', 'superviseur', 'group_prix', 'week_1', 'week_2', 'Mon', 'Tue', 'Wed', 'Thu',
#        'Sat', 'Sun']
# total = total[feat_total]
df_total = pd.concat([df_vente,total])
df_total.tail()

,agent,superviseur,group_prix,zone,week_1,week_2,week_3,Mon,Tue,Wed,Thu,Sat,Sun
147,YVES PATRICE DJILAMBA BASSENE,OUSMANE NIANG,Total,NaN,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
148,Yacouba Fomba,Samba Coulibaly,Total,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149,Yacouba Kienou,N. Jean Paul BAYALA,Total,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
150,Yaya Konaré,Samba Coulibaly,Total,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
151,Zinta Boris TIRA,Lacina Oura,Total,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
day_sales = pd.pivot_table(df_vente_mois[df_vente_mois['Weeks']==week_int2(date.today())],values='ventes',index=['agent','superviseur','group_prix','zone'],columns='Day', aggfunc=np.sum).reset_index()
day_sales.fillna(0,inplace=True)

In [38]:
day_sales

Day,agent,superviseur,group_prix,zone,Mon,Tue
0,ABDOU RAZACK BALDE,IBRAHIMA THIONGANE,WowSolar60_Mensuelle 8500/4250,KOLDA,1.0,0.0
1,ABDOURAHMANI ZIO,Y. Clément YONLI,WowSolarTV32 - Cash - BF,Zone Nahouri,1.0,0.0
2,AMADOU SOMBIE,Sahiba DIAO,WowSolar60 - Annuel - BF,Zone Nayala,4.0,1.0
3,Abdallah OUATTARA,Sié Tanguy Kader KAM,WowSolarTV32 - Mensuel - BF,Zone GAOUA,1.0,0.0
4,Adjaratou KABORE,D. Abdias TARNAGDA,WowSolar60 - Annuel - BF,Zone Tenkodogo,2.0,0.0
5,Alered DA,Sié Tanguy Kader KAM,WowSolar60 - Mensuel - BF,Zone GAOUA,1.0,0.0
6,Alered DA,Sié Tanguy Kader KAM,WowSolarTV32 - Mensuel - BF,Zone GAOUA,0.0,1.0
7,BAH KEITA,DEMBA GUEYE,WowSolar60_Mensuelle 8500/4250,KEDOUGOU,1.0,0.0
8,Bagadian Keita,David Timbine,WowSolar60 - Annuel - ML,KAYES,1.0,0.0
9,Chaka Togola,Samba Coulibaly,WowSolar60 - Annuel - ML,KOULIKORO,1.0,0.0


In [39]:
day_sales.columns

Index(['agent', 'superviseur', 'group_prix', 'zone', 'Mon', 'Tue'], dtype='object', name='Day')

In [40]:
df_backlog.head()

,agent_id,closed_date,agent,type_ticket,annee,mois,jour,status,ticket
0,2185,2019-11-05,IBRAHIMA TOURE,Reposséder,2019,11,5,cloturer,1
1,261,2019-11-13,HAMARO CISSE,Ticket technique,2019,11,13,cloturer,2
2,2287,2019-09-25,Allassane SAWADOGO,Reposséder,2019,9,25,cloturer,1
3,277,2019-06-25,IBOU CISSE,Reposséder,2019,6,25,customer_paid,1
4,291,2021-02-23,Bagadian Keita,Reposséder,2021,2,23,customer_paid,1


In [41]:
df_backlog_sup = pd.merge(df_backlog,df_ag_sup[['agent_id','superviseur','zone']], how= 'inner', on= 'agent_id')
df_backlog_sup.head()

,agent_id,closed_date,agent,type_ticket,annee,mois,jour,status,ticket,superviseur,zone
0,2185,2019-11-05,IBRAHIMA TOURE,Reposséder,2019,11,5,cloturer,1,ALIOU YADE,DIOURBEL
1,2185,2019-05-23,IBRAHIMA TOURE,Reposséder,2019,5,23,customer_paid,1,ALIOU YADE,DIOURBEL
2,2185,2019-10-02,IBRAHIMA TOURE,Ticket technique,2019,10,2,cloturer,1,ALIOU YADE,DIOURBEL
3,2185,2019-08-13,IBRAHIMA TOURE,Reposséder,2019,8,13,customer_paid,1,ALIOU YADE,DIOURBEL
4,2185,2019-10-16,IBRAHIMA TOURE,Ticket technique,2019,10,16,cloturer,1,ALIOU YADE,DIOURBEL


In [42]:
df_backlog_sup['Weeks'] = df_backlog_sup['closed_date'].apply(week_int2)
df_backlog_sup['Month'] = df_backlog_sup['closed_date'].apply(lambda x: x.strftime('%b'))
df_backlog_sup['Day'] = df_backlog_sup['closed_date'].apply(lambda x: x.strftime('%a'))
df_backlog_sup.head()

,agent_id,closed_date,agent,type_ticket,annee,mois,jour,status,ticket,superviseur,zone,Weeks,Month,Day
0,2185,2019-11-05,IBRAHIMA TOURE,Reposséder,2019,11,5,cloturer,1,ALIOU YADE,DIOURBEL,week_2,Nov,Tue
1,2185,2019-05-23,IBRAHIMA TOURE,Reposséder,2019,5,23,customer_paid,1,ALIOU YADE,DIOURBEL,week_4,May,Thu
2,2185,2019-10-02,IBRAHIMA TOURE,Ticket technique,2019,10,2,cloturer,1,ALIOU YADE,DIOURBEL,week_1,Oct,Wed
3,2185,2019-08-13,IBRAHIMA TOURE,Reposséder,2019,8,13,customer_paid,1,ALIOU YADE,DIOURBEL,week_3,Aug,Tue
4,2185,2019-10-16,IBRAHIMA TOURE,Ticket technique,2019,10,16,cloturer,1,ALIOU YADE,DIOURBEL,week_3,Oct,Wed


In [43]:
feature1 = ['closed_date','annee','mois','agent','superviseur','status','type_ticket','zone','Weeks','Month','Day','ticket']
df_backlog_mois = df_backlog_sup[feature1][(df_backlog_sup['annee']==date.today().year)&(df_backlog_sup['mois']==date.today().month)]
df_backlog_mois.head()

,closed_date,annee,mois,agent,superviseur,status,type_ticket,zone,Weeks,Month,Day,ticket
61,2021-02-01,2021,2,Allassane SAWADOGO,Emmanuel KABRE,cloturer,Reposséder,Zone houndé,week_1,Feb,Mon,1
74,2021-02-10,2021,2,Allassane SAWADOGO,Emmanuel KABRE,cloturer,Ticket technique,Zone houndé,week_2,Feb,Wed,1
81,2021-02-22,2021,2,Allassane SAWADOGO,Emmanuel KABRE,cloturer,Reposséder,Zone houndé,week_4,Feb,Mon,1
85,2021-02-01,2021,2,Allassane SAWADOGO,Emmanuel KABRE,cloturer,Ticket technique,Zone houndé,week_1,Feb,Mon,1
110,2021-02-22,2021,2,Allassane SAWADOGO,Emmanuel KABRE,customer_paid,Reposséder,Zone houndé,week_4,Feb,Mon,1


In [44]:
week_backlog = pd.pivot_table(df_backlog_mois,values='ticket',index=['agent','type_ticket','status'],columns='Weeks', aggfunc=np.sum, margins=True, margins_name='Total')
week_backlog.fillna(0,inplace=True)
week_backlog=week_backlog.reset_index()

In [45]:
week_backlog[week_backlog['status']=='à faire']

Weeks,agent,type_ticket,status,week_1,week_2,week_3,week_4,Total


In [46]:
# import schedule
# import calendar
# import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np
# import dash
# import dash_bootstrap_components as dbc
# import dash_html_components as html
# import dash_core_components as dcc
# import plotly.express as px
# from dash.dependencies import Input, Output
# import dash_table as dt
# import  psycopg2
# from psycopg2 import Error
# from sqlalchemy import text
# from datetime import date
# import schedule

# query2 = open('Scripts/query2.sql')
# stock = open('Scripts/user_stock.sql')
# sup = open('Scripts/Supervisor.sql')
# vente = open('Scripts/ventes.sql')
# sql_text1 = query2.read()
# sql_text2 = stock.read()
# sql_text3 = vente.read()
# sql_text4 = sup.read()

# # Connection to oolusolar database
# try:
#     connection = psycopg2.connect(user = 'chartio_read_only_user',
#                                   password = '2ZVF01USUWTKV3K9JJFY',
#                                   host = 'oolu-main-postgresql.cfa4plgxjs0u.eu-central-1.rds.amazonaws.com',
#                                   port ='5432',
#                                   database = 'oolusolar_analytics')
#     cursor = connection.cursor()
#     cursor.execute("SELECT version();")
#     record = cursor.fetchone()
#     print('You are Successfully connected to - ',record, '\n')
# except (Exception, Error) as error:
#     print(" Connection failed, try again", error)
#     cursor.close()

# df_ag_sup  = pd.read_sql_query(sql_text1 , connection)
# df_stock  = pd.read_sql_query(sql_text2 , connection)
# df_vente  = pd.read_sql_query(sql_text3 , connection)
# df_sup  = pd.read_sql_query(sql_text4 , connection)
# cursor.close()
# connection.close()


# table_header_style = {
#     "backgroundColor": "rgb(2,21,70)",
#     "color": "white",
#     "textAlign": "center",
#     'fontSize': 20
# }

# df_vente['created_at']= pd.to_datetime(df_vente['created_at'])
# df_ag_sup['date_debut']= pd.to_datetime(df_ag_sup['date_debut'])
# df_sup_stock = pd.merge(df_sup,df_stock, how= 'left', on= 'agent_id')

# #======================================== Supervieur =================================================================

# sup_feat = ['superviseur','zone','stock']
# df_sup_stock = df_sup_stock[sup_feat]
# supervisor = df_sup_stock.groupby(['superviseur','zone']).sum().reset_index()

# #======================================= Ventes mensuelles des Agents ===============================================================

# df_vente_agent = pd.merge(df_vente,df_ag_sup[['agent_id','superviseur','zone']], how= 'inner', on= 'agent_id')

# def week_int2(x):
#     y =x.year
#     m = x.month
#     d = x.day
#     list_week = calendar.monthcalendar(y,m)
#     for i in range(len(list_week)):
#         if d in list_week[i]:
#             return 'week_'+str(i+1)

# df_vente_agent['Weeks'] = df_vente_agent['created_at'].apply(week_int2)
# df_vente_agent['Month'] = df_vente_agent['created_at'].apply(lambda x: x.strftime('%b'))
# df_vente_agent['Day'] = df_vente_agent['created_at'].apply(lambda x: x.strftime('%a'))
# df_vente_agent['CA'] = df_vente_agent['deposit']*df_vente_agent['ventes']

# feature = ['created_at','annee','mois','agent','group_prix','superviseur','zone','Weeks','Month','Day','CA','ventes']
# df_vente_mois = df_vente_agent[feature][(df_vente_agent['annee'] == date.today().year)&(df_vente_agent['mois']==date.today().month)]

# #==================================================  Week Sales ====================================================================
# week_sales = pd.pivot_table(df_vente_mois,values='ventes',index=['agent','superviseur','group_prix','zone'],columns='Weeks', aggfunc=np.sum).reset_index()
# week_sales.fillna(0,inplace=True)

# #==================================================  Daye Sales ====================================================================

# day_sales = pd.pivot_table(df_vente_mois[df_vente_mois['Weeks']==week_int2(date.today())],values='ventes',index=['agent','superviseur','group_prix','zone'],columns='Day', aggfunc=np.sum).reset_index()
# day_sales.fillna(0,inplace=True)

# week_sale_col = list(week_sales.columns)
# day_sale_col = list(day_sales.columns)



# app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# app.layout = html.Div([
#                         dbc.Row([
#                             dbc.Col(
#                                 html.Div([
#                                         html.Img(src=app.get_asset_url('oolu.png'),
#                                              id='oolu-logo',
#                                              style={
#                                                  "height": "60px",
#                                                  "width": "auto",
#                                                  "margin-bottom": "25px",},
#                                      )],className="one-third column",
#                                 ),width={'size': 1}),
#                             dbc.Col([
#                                 html.H1('Supervison Des Agents',
#                                         style={
#                                             'font-size': '450%',
#                                             'textAlign': 'center',
#                                             'color': 'rgb(255,216,0)'
#                                             }),
#                             ],width={'size': 5, 'offset': 3},)
#                         ]),
#                         html.Br(),
#                         dbc.Row([
#                             dbc.Col([dcc.Dropdown(id = 'sup', value='KHASSIM  DIOP',multi=False,
#                                options=[{'label':x, 'value':x} for x in df_sup_stock['superviseur'].unique()],
#                                 className='form-dropdown',
#                              ),],width={'size': 3, 'offset':9})
#                         ]),
#                         html.Br(),
#                         html.Div([
#                             dbc.Row([
#                                 dbc.Col([
#                                     dt.DataTable(
#                                         id = 'sup_stock',
#                                         style_header=table_header_style,
#                                         columns= [{'name':i, 'id': i} for i in sup_feat],
#                                         # data=df1.to_dict('records'),
#                                         style_table={'border-collapse': 'collapse','height': '100px', 'overflowY': 'auto'}

#                                     )
#                                 ],width={'size': 3,'offset':3})
#                             ]),
#                             dbc.Row([
#                                 dbc.Col([html.H6(children='Agent Sales',
#                                                 style={
#                                                     'textAlign': 'center',
#                                                     'color': 'white'}
#                                                 ),
#                                     dt.DataTable(
#                                         id = 'week_sales',
#                                         style_header=table_header_style,
#                                         columns= [{'name':i, 'id': i} for i in week_sale_col],
#                                         # data=df1.to_dict('records'),
#                                         style_table={'height': '600px', 'overflowY': 'auto'}

#                                     )
#                                 ],width={'size': 6,'offset':1})
#                             ]),

#                         ])
# ])


# @app.callback(
#     Output('sup_stock', 'data'),
#     Input('sup', 'value'))
# def update_table(select_sup):
#     data_table1 = supervisor[supervisor['superviseur'] == select_sup]
#     return data_table1.to_dict('records')
# @app.callback(
#     Output('agent_sup', 'data'),
#     Input('sup', 'value'))
# def update_table(select_sup):
#     data_table2 = week_sales[week_sales['superviseur'] == select_sup]
#     data_table2 = data_table2[week_sale_col]
#     data_table3 = pd.pivot_table(data_table2,values='stock',index=['agent','agent_id'],columns='category', aggfunc=np.sum).reset_index()
#     data_table3.fillna(0,inplace=True)
#     print(data_table3.head())
#     data_table3['Total Stock'] = data_table3['SHS+TV']+data_table3['pico'] +data_table3['accessories']

#     return data_table3.to_dict('records')


# if __name__ == '__main__':
#     app.run_server(debug=True, host='0.0.0.0', port=8082)

In [47]:
df_ticket_sup = pd.merge(df_ticket,df_ag_sup[['agent_id','superviseur','zone']], how= 'inner', on= 'agent_id')
df_ticket_sup['Weeks'] = df_ticket_sup['closed_date'].apply(week_int2)
df_ticket_sup['Month'] = df_ticket_sup['closed_date'].apply(lambda x: x.strftime('%b'))
df_ticket_sup['Day'] = df_ticket_sup['closed_date'].apply(lambda x: x.strftime('%a'))

feature1 = ['closed_date','annee','mois','agent_id','agent','superviseur','status','type_ticket','zone','Weeks','Month','Day','ticket']
df_ticket_mois = df_ticket_sup[feature1][(df_ticket_sup['annee']==date.today().year)&(df_ticket_sup['mois']==date.today().month)]

In [48]:
week_ticket = pd.pivot_table(df_ticket_mois,values='ticket',index=['agent_id','agent','superviseur','type_ticket','status'],columns='Weeks', aggfunc=np.sum).reset_index()
week_ticket.fillna(0,inplace=True)
week_ticket_col = list(week_ticket.columns)
# print(week_ticket_col)
week_ticket.head()

Weeks,agent_id,agent,superviseur,type_ticket,status,week_1,week_2,week_3,week_4
0,45,ALPHA BABOU,Ndeye Saly Faye,Reposséder,customer_paid,2.0,3.0,0.0,0.0
1,45,ALPHA BABOU,Ndeye Saly Faye,Ticket technique,cloturer,1.0,0.0,4.0,3.0
2,53,AMADOU BOCAR SAGNA,SADIA TAMBA,Ticket technique,cloturer,0.0,0.0,1.0,0.0
3,82,Abdoulaye Bamako Coulibaly,Samba Coulibaly,Reposséder,cloturer,0.0,0.0,3.0,0.0
4,82,Abdoulaye Bamako Coulibaly,Samba Coulibaly,Reposséder,customer_paid,0.0,1.0,1.0,0.0


In [49]:
feat_ticket = ['agent_id', 'type_ticket', 'status','Mon','Tue', 'Wed','Thu', 'Sat', 'Sun']

In [50]:
day_ticket = pd.pivot_table(df_ticket_mois,values='ticket',index=['agent_id','agent','superviseur','type_ticket','status','Weeks'],columns='Day', aggfunc=np.sum).reset_index()
day_ticket.fillna(0,inplace=True)
day_ticket.head()

Day,agent_id,agent,superviseur,type_ticket,status,Weeks,Fri,Mon,Sat,Sun,Thu,Tue,Wed
0,45,ALPHA BABOU,Ndeye Saly Faye,Reposséder,customer_paid,week_1,0.0,0.0,0.0,0.0,0.0,2.0,0.0
1,45,ALPHA BABOU,Ndeye Saly Faye,Reposséder,customer_paid,week_2,0.0,1.0,0.0,0.0,0.0,1.0,1.0
2,45,ALPHA BABOU,Ndeye Saly Faye,Ticket technique,cloturer,week_1,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,45,ALPHA BABOU,Ndeye Saly Faye,Ticket technique,cloturer,week_3,0.0,4.0,0.0,0.0,0.0,0.0,0.0
4,45,ALPHA BABOU,Ndeye Saly Faye,Ticket technique,cloturer,week_4,0.0,0.0,0.0,0.0,0.0,3.0,0.0


In [51]:
df_day_ticket = day_ticket[feat_ticket][day_ticket['Weeks']==week_int2(date.today())]

df_ticket = pd.merge(week_ticket,df_day_ticket, how= 'left', on= ['agent_id','type_ticket','status'])
df_ticket.fillna(0,inplace=True)
df_ticket = df_ticket.drop([week_int2(date.today()),'agent_id'],axis=1)

In [52]:
df_ticket.head()

,agent,superviseur,type_ticket,status,week_1,week_2,week_3,Mon,Tue,Wed,Thu,Sat,Sun
0,ALPHA BABOU,Ndeye Saly Faye,Reposséder,customer_paid,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ALPHA BABOU,Ndeye Saly Faye,Ticket technique,cloturer,1.0,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0
2,AMADOU BOCAR SAGNA,SADIA TAMBA,Ticket technique,cloturer,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Abdoulaye Bamako Coulibaly,Samba Coulibaly,Reposséder,cloturer,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Abdoulaye Bamako Coulibaly,Samba Coulibaly,Reposséder,customer_paid,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


## Stock agents

In [53]:
df_ag_stock1 = pd.merge(df_ag_sup,df_stock, how= 'inner', on= 'agent_id')

In [54]:
df_ag_stock1.head()

,agent_id,date_debut,username,agent,agent_role,role_id,zone_id,zone,id_tenant,superviseur,sup_username,sup_id,stock_name,stock_type,product_type,category,stock
0,3761,2021-02-01,badini.issiaka,Issiaka Badini,Agent Polivalent,5,22,Zone Centre,4,Bépian GUEL,guel.bepian,2558,Issiaka Badini - badini.issiaka,personal,Fan component,None,1
1,3761,2021-02-01,badini.issiaka,Issiaka Badini,Agent Polivalent,5,22,Zone Centre,4,Bépian GUEL,guel.bepian,2558,Issiaka Badini - badini.issiaka,personal,WowSolar Controller,pico,3
2,3675,2020-12-03,cisse.mamadou,Mamadou CISSE,Agent Polivalent,5,15,Zone dedougou,4,Lacina Oura,lacina.oura,431,Mamadou CISSE - cisse.mamadou,personal,WowSolar Controller,pico,1
3,1347,2018-12-03,honore.ilboudo,Honore Ilboudo,Agent Commercial,14,22,Zone Centre,4,Bépian GUEL,guel.bepian,2558,Honore Ilboudo - honore.ilboudo,personal,Radio component,pico,1
4,1347,2018-12-03,honore.ilboudo,Honore Ilboudo,Agent Commercial,14,22,Zone Centre,4,Bépian GUEL,guel.bepian,2558,Honore Ilboudo - honore.ilboudo,personal,WowSolar Controller,pico,5


In [55]:
stock_feat = ['agent','superviseur','category','product_type','stock']
df_ag_stock1=df_ag_stock1[stock_feat]
df_ag_stock1.head()

,agent,superviseur,category,product_type,stock
0,Issiaka Badini,Bépian GUEL,None,Fan component,1
1,Issiaka Badini,Bépian GUEL,pico,WowSolar Controller,3
2,Mamadou CISSE,Lacina Oura,pico,WowSolar Controller,1
3,Honore Ilboudo,Bépian GUEL,pico,Radio component,1
4,Honore Ilboudo,Bépian GUEL,pico,WowSolar Controller,5


In [56]:
print(f' category of stocks : \n {df_ag_stock1.category.unique()}')
print(f' Type of product in stock : \n {df_ag_stock1.product_type.unique()}')

 category of stocks : 
 [None 'pico' 'SHS+TV' 'accessories']
 Type of product in stock : 
 ['Fan component' 'WowSolar Controller' 'Radio component' 'SKH Controller'
 'WowSolar Extension cable' 'WowSolar Lamp 140lm' 'WowSolar Panel 6W'
 'WowSolar USB charger' 'WowSolar Wall mount' 'SKH Extension cable'
 'SKH Lamp' 'SKH PV Panel' 'SKH USB cable' 'SKH Wall mount'
 'LNB Head incl. support' 'Satellite Cable 12m incl. connectors'
 'Satellite Dish incl. support' 'WowSolar Batterie GM 80W'
 'WowSolar Lamp 400lm' 'WowSolar Panel 40W' 'WowSolar Remote control'
 'WowSolar TV Screen 19 ' 'D100 Controller' 'D100 Extension cable'
 'D100 Lamp 120lm' 'D100 Lamp 220lm' 'D100 Panel' 'Hairclipper component'
 'D100 Multifunction charger' 'WowSolar TV Screen 22'
 'WowSolar TV Screen 32']


In [57]:
df_ag_stock1[df_ag_stock1['category'].isin([None])]

,agent,superviseur,category,product_type,stock
0,Issiaka Badini,Bépian GUEL,None,Fan component,1
5,Oumar Doumbia,Hamaye Makanguile,None,Fan component,1
29,WOUROKOLE COULIBALY,Lacina Oura,None,Fan component,1
46,PATHE WATHIE,SOULEYMANE MANE,None,Fan component,2
113,Adama Barry,Emmanuel KABRE,None,WowSolar Remote control,1
...,...,...,...,...,...
936,AMADOU SOMBIE,Sahiba DIAO,None,Fan component,3
949,TALLA FAYE,THIERNO FAYE,None,Fan component,1
959,MARITEW CHIMER GUEYE,OUSMANE NIANG,None,Fan component,3
975,Abdoulaye Keita,David Timbine,None,Fan component,2


In [58]:
df_ag_stock1['Type de produit'] = [df_ag_stock1['product_type'].iloc[i] if df_ag_stock1['category'].iloc[i] in [None]
                                   else df_ag_stock1['category'].iloc[i] for i in range(df_ag_stock1.shape[0]) ]

In [59]:
df_ag_stock1.head()

,agent,superviseur,category,product_type,stock,Type de produit
0,Issiaka Badini,Bépian GUEL,None,Fan component,1,Fan component
1,Issiaka Badini,Bépian GUEL,pico,WowSolar Controller,3,pico
2,Mamadou CISSE,Lacina Oura,pico,WowSolar Controller,1,pico
3,Honore Ilboudo,Bépian GUEL,pico,Radio component,1,pico
4,Honore Ilboudo,Bépian GUEL,pico,WowSolar Controller,5,pico


In [78]:
stocks = df_ag_stock1.groupby(['agent','superviseur','Type de produit']).sum().reset_index()
stocks.head()

,agent,superviseur,Type de produit,stock
0,ABDOU RAZACK BALDE,IBRAHIMA THIONGANE,pico,2
1,ABDOULAYE KANTE,IBRAHIMA THIONGANE,pico,3
2,ABDOULAYE MBACKE DIAW,ALIOU YADE,pico,26
3,ABDOURAHMANI ZIO,Y. Clément YONLI,pico,2
4,ABIB SARR,IBRAHIMA BAILIBORE TENDENG,pico,3


In [79]:
stocks.loc[:,'total'] = 10

In [80]:
stocks.head()

,agent,superviseur,Type de produit,stock,total
0,ABDOU RAZACK BALDE,IBRAHIMA THIONGANE,pico,2,10
1,ABDOULAYE KANTE,IBRAHIMA THIONGANE,pico,3,10
2,ABDOULAYE MBACKE DIAW,ALIOU YADE,pico,26,10
3,ABDOURAHMANI ZIO,Y. Clément YONLI,pico,2,10
4,ABIB SARR,IBRAHIMA BAILIBORE TENDENG,pico,3,10
